# Relatório 2

**Nome:** Thiago Lopes <br>
**Matrícula:** 20100358 <br>
**Turma:** T2

# Bibliotecas python utilizadas

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit


# Funçoes Auxiliares

In [ ]:
def exp_model(x, a, b):
    return a * b ** x

def power_model(x, a, b):
    return a * x ** b


In [ ]:
def grafico_mmq(params_exp, params_power):
    # Valores preditos para plotagem das curvas
    x_pred = np.linspace(0, 7, 100)
    y_pred_exp = exp_model(x_pred, *params_exp)
    y_pred_power = power_model(x_pred, *params_power)

    # Plotar o diagrama de dispersão e as curvas de ajuste
    plt.figure(figsize=(8, 6))
    plt.scatter(x, y, color='blue', label='Dados')
    plt.plot(x_pred, y_pred_exp, color='green', label='Ajuste Exponencial')
    plt.plot(x_pred, y_pred_power, color='red', label='Ajuste de Potência')
    plt.title('Diagrama de Dispersão e Curvas de Ajuste')
    plt.xlabel('Horas (x)')
    plt.ylabel('Número de Bactérias por Volume Unitário (y)')
    plt.grid(True)
    plt.legend()
    plt.show()
    

# Métodos implementados: 

In [ ]:
def lagrange_interpolation(x, y, z):
    m = len(x)
    Pz = 0

    for i in range(m):
        c = 1
        d = 1
        for j in range(m):
            if i != j:
                c *= (z - x[j])
                d *= (x[i] - x[j])
        Pz += y[i] * (c / d)

    return Pz


In [ ]:
def newton_interpolation(x, y, z):
    m = len(x)
    Dely = y.copy()

    # Construção das diferenças divididas
    for i in range(1, m):
        for k in range(m-1, i-1, -1):
            Dely[k] = (Dely[k] - Dely[k-1]) / (x[k] - x[k-i])

    # Avaliação do polinômio pelo método de Horner
    Pz = Dely[-1]
    for i in range(m-2, -1, -1):
        Pz = Pz * (z - x[i]) + Dely[i]

    return Pz


In [ ]:
def spline_cubica_natural(x, y):
    n = len(x)
    h = np.diff(x)
    alpha = np.zeros(n)
    for i in range(1, n-1):
        alpha[i] = 3 * (y[i+1] - y[i]) / h[i] - 3 * (y[i] - y[i-1]) / h[i-1]
    
    l = np.zeros(n)
    u = np.zeros(n)
    z = np.zeros(n)
    l[0] = 1
    u[0] = 0
    z[0] = 0
    
    for i in range(1, n-1):
        l[i] = 2 * (x[i+1] - x[i-1]) - h[i-1] * u[i-1]
        u[i] = h[i] / l[i]
        z[i] = (alpha[i] - h[i-1] * z[i-1]) / l[i]
    
    l[n-1] = 1
    z[n-1] = 0
    c = np.zeros(n)
    b = np.zeros(n)
    d = np.zeros(n)
    
    for j in range(n-2, -1, -1):
        c[j] = z[j] - u[j] * c[j+1]
        b[j] = (y[j+1] - y[j]) / h[j] - h[j] * (c[j+1] + 2 * c[j]) / 3
        d[j] = (c[j+1] - c[j]) / (3 * h[j])
    
    return y, b, c, d

def encontrar_latitude_spline(x, y, x_interp):
    y, b, c, d = spline_cubica_natural(x, y)
    n = len(x)
    for i in range(n-1):
        if x[i] <= x_interp <= x[i+1]:
            xi = x[i]
            yi = y[i]
            hi = x[i+1] - x[i]
            interpolated_value = yi + b[i] * (x_interp - xi) + c[i] * (x_interp - xi)**2 + d[i] * (x_interp - xi)**3
            return interpolated_value
    
    # Se não encontrou o intervalo adequado (o que não deveria acontecer neste caso), retornar None
    return None

def encontrar_extremo_latitude(x, y):
    y, b, c, d = spline_cubica_natural(x, y)
    n = len(x)
    
    # Encontrar o índice do ponto de latitude máxima
    index_max_latitude = np.argmax(y)
    
    # Retornar as coordenadas desse ponto
    longitude_max_latitude = x[index_max_latitude]
    latitude_max = y[index_max_latitude]
    
    return longitude_max_latitude, latitude_max


In [ ]:
def ajustar_bacterias(x, y):

    # Ajuste dos dados para y = ab^x
    params_exp, _ = curve_fit(exp_model, x, y)
    a_exp, b_exp = params_exp
    y_exp_fit = exp_model(x, *params_exp)

    # Ajuste dos dados para y = ax^b (excluindo x=0)
    x_nonzero = x[1:]
    y_nonzero = y[1:]

    params_power, _ = curve_fit(power_model, x_nonzero, y_nonzero)
    a_power, b_power = params_power
    y_power_fit = power_model(x_nonzero, *params_power)

    return (a_exp, b_exp), (a_power, b_power)


# Exercícios

## Exercício 1

In [ ]:
# Dados fornecidos
tempos = [0, 10, 30, 60, 90, 120, 140]  # tempo em minutos
distancias = [0.00, 8.00, 27.00, 58.00, 100.00, 145.00, 160.00]  # distância percorrida em km

# Usando os quatro primeiros pontos para calcular a distância percorrida em 45 minutos
tempos_4 = [0, 10, 30, 60]
distancias_4 = [0.00, 8.00, 27.00, 58.00]

# -----------------------------
# lagrange_interpolation
# -----------------------------
print("Lagrange Interpolation")
# A)
distancia_45_min = lagrange_interpolation(tempos_4, distancias_4, 45)
print(f"Distância percorrida nos primeiros 45 minutos: {distancia_45_min:.2f} km")

# B)
tempo_para_metade = lagrange_interpolation(distancias, tempos, 80)
print(f"Tempo necessário para percorrer metade do caminho (80 km): {tempo_para_metade:.2f} minutos")

print("\n")

# -----------------------------
# newton_interpolation
# -----------------------------
print("Newton Interpolation")
# A)
distancia_45_min = newton_interpolation(tempos_4, distancias_4, 45)
print(f"Distância percorrida nos primeiros 45 minutos: {distancia_45_min:.2f} km")

# B)
tempo_para_metade = newton_interpolation(distancias, tempos, 80)
print(f"Tempo necessário para percorrer metade do caminho (80 km): {tempo_para_metade:.2f} minutos")

## Exercício 2

In [ ]:
# Dados fornecidos
comprimentos = [500, 1000, 1500, 2000, 2500, 3000, 3500, 4000]
resistencias = [2.74, 5.48, 7.90, 11.00, 13.93, 16.43, 20.24, 23.52]

# -----------------------------
# lagrange_interpolation
# -----------------------------
print("Lagrange Interpolation")
# Interpolação de grau 2 para 1730 m
comprimentos_grau_2 = [1500, 2000, 2500]
resistencias_grau_2 = [7.90, 11.00, 13.93]
resistencia_1730_g2 = lagrange_interpolation(comprimentos_grau_2, resistencias_grau_2, 1730)
print(f"Resistência para 1730 m (grau 2): {resistencia_1730_g2:.2f} Ohms")

# Interpolação de grau 2 para 3200 m
comprimentos_grau_2 = [3000, 3500, 4000]
resistencias_grau_2 = [16.43, 20.24, 23.52]
resistencia_3200_g2 = lagrange_interpolation(comprimentos_grau_2, resistencias_grau_2, 3200)
print(f"Resistência para 3200 m (grau 2): {resistencia_3200_g2:.2f} Ohms")

# Interpolação de grau 3 para 1730 m
comprimentos_grau_3 = [1000, 1500, 2000, 2500]
resistencias_grau_3 = [5.48, 7.90, 11.00, 13.93]
resistencia_1730_g3 = lagrange_interpolation(comprimentos_grau_3, resistencias_grau_3, 1730)
print(f"Resistência para 1730 m (grau 3): {resistencia_1730_g3:.2f} Ohms")

# Interpolação de grau 3 para 3200 m
comprimentos_grau_3 = [2500, 3000, 3500, 4000]
resistencias_grau_3 = [13.93, 16.43, 20.24, 23.52]
resistencia_3200_g3 = lagrange_interpolation(comprimentos_grau_3, resistencias_grau_3, 3200)
print(f"Resistência para 3200 m (grau 3): {resistencia_3200_g3:.2f} Ohms")

print("\n")

# -----------------------------
# newton_interpolation
# -----------------------------
print("Newton Interpolation")
# Interpolação de grau 2 para 1730 m
comprimentos_grau_2 = [1500, 2000, 2500]
resistencias_grau_2 = [7.90, 11.00, 13.93]
resistencia_1730_g2 = newton_interpolation(comprimentos_grau_2, resistencias_grau_2, 1730)
print(f"Resistência para 1730 m (grau 2): {resistencia_1730_g2:.2f} Ohms")

# Interpolação de grau 2 para 3200 m
comprimentos_grau_2 = [3000, 3500, 4000]
resistencias_grau_2 = [16.43, 20.24, 23.52]
resistencia_3200_g2 = newton_interpolation(comprimentos_grau_2, resistencias_grau_2, 3200)
print(f"Resistência para 3200 m (grau 2): {resistencia_3200_g2:.2f} Ohms")

# Interpolação de grau 3 para 1730 m
comprimentos_grau_3 = [1000, 1500, 2000, 2500]
resistencias_grau_3 = [5.48, 7.90, 11.00, 13.93]
resistencia_1730_g3 = newton_interpolation(comprimentos_grau_3, resistencias_grau_3, 1730)
print(f"Resistência para 1730 m (grau 3): {resistencia_1730_g3:.2f} Ohms")

# Interpolação de grau 3 para 3200 m
comprimentos_grau_3 = [2500, 3000, 3500, 4000]
resistencias_grau_3 = [13.93, 16.43, 20.24, 23.52]
resistencia_3200_g3 = newton_interpolation(comprimentos_grau_3, resistencias_grau_3, 3200)
print(f"Resistência para 3200 m (grau 3): {resistencia_3200_g3:.2f} Ohms")


## Exercício 3

In [ ]:
# Dados do problema
x = np.array([5.6, 5.93, 6.26, 6.6])  # longitude em graus
y = np.array([26.93, 26.84, 27.04, 26.97])  # latitude em graus

# A)
# Longitude desejada para interpolação
longitude_desejada = 6.0

# Interpolação para encontrar a latitude correspondente
latitude_interpolada = encontrar_latitude_spline(x, y, longitude_desejada)

if latitude_interpolada is not None:
    print(f"A latitude correspondente à longitude {longitude_desejada} é: {latitude_interpolada:.2f} graus.\n")
else:
    print("Não foi possível interpolar a latitude para a longitude desejada.\n")
    
    
    
# B)
# Intervalo de longitude desejado para encontrar o ponto mais distante do equador
intervalo_inicio = 6.26
intervalo_fim = 6.6

# Filtrar os dados dentro do intervalo de longitude desejado
indices_intervalo = np.where((x >= intervalo_inicio) & (x <= intervalo_fim))[0]

if len(indices_intervalo) > 0:
    x_intervalo = x[indices_intervalo]
    y_intervalo = y[indices_intervalo]

    # Encontrar o ponto de latitude máxima dentro do intervalo
    longitude_max_latitude, latitude_max = encontrar_extremo_latitude(x_intervalo, y_intervalo)

    # Resultado
    print(f"As coordenadas da estrada no ponto mais distante do equador são:")
    print(f"Longitude: {longitude_max_latitude:.2f} graus")
    print(f"Latitude: {latitude_max:.2f} graus")
else:
    print("Não há pontos dentro do intervalo especificado de longitude.\n")


## Exercício 4

In [ ]:
# Dados fornecidos
x = np.array([0, 1, 2, 3, 4, 5, 6])
y = np.array([32, 47, 65, 92, 132, 190, 275])

# Ajustar os dados
params_exp, params_power = ajustar_bacterias(x, y)

print(f"Ajuste para y = ab^x: a = {params_exp[0]:.4f}, b = {params_exp[1]:.4f}")
print(f"Ajuste para y = ax^b: a = {params_power[0]:.4f}, b = {params_power[1]:.4f}\n")


# Prever o número de bactérias em 7 horas
x_pred = 7

# Exp
a, b = params_exp[0], params_exp[1]
y_pred = a * b ** x_pred

# # Power
# a, b = params_power[0], params_power[1]
# y_pred = a * x_pred ** b

print(f"Previsão para 7 horas: y = {y_pred:.4f}")


grafico_mmq(params_exp, params_power)
